PPCA lavora con più variabili osservabili anzichè una sola

In [1]:
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import aesara.tensor as at
import arviz as az
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, r2_score
from scipy import stats
import scipy

WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
scaler = StandardScaler()

In [3]:
data = np.load('prova.npy', allow_pickle=True).item()

## test with new data

In [4]:
import deepemogp.feature_extractor as feature_extractor
import deepemogp.signal.physio as physio
import deepemogp.signal.face as face
import deepemogp.signal.annotation as annotation
import deepemogp.datasets as datasets
import deepemogp.signal.behavior as behavior

# from: https://github.com/SheffieldML/GPy
#import GPy
# from: https://github.com/SheffieldML/PyDeepGP
#import deepgp

import pandas as pd
import numpy as np

show = False
# definition of the feature extractors to be used later
f1 = feature_extractor.FE('wavelet')
f2 = feature_extractor.FE('wavelet', window=(2, 1))
f3 = feature_extractor.FE('mean', window=(2, 1))

# definition of the physiological signals to be extracted
eda = physio.EDA(f3)
hr = physio.HR(f3)

pupil = behavior.PUPIL(f3)

# definition of the emotional annotation to be extracted
#va = annotation.VA('valence', f3)
#ar = annotation.VA('arousal', f3)
# extraction of the desired data from the dataset
d = datasets.FEAR(signals={hr, pupil}, subjects={'2_1'})

for s in d.signals:
    # preprocess ...
    #s.preprocess(show=show)

    # ... and extract features from each signal type
    s.feature_ext.extract_feat_without_preprocess(s, show=show)

pupil_data = list(d.signals)[0].features
hr_data = list(d.signals)[1].features

print(pupil_data)
print(hr_data)

>> Loading HR for subject 2 and session 1 from dataset FEAR
[{'data': [-0.0598145, -0.0598145, -0.059967, -0.0601196, -0.0604248, -0.0602722, -0.0605774, -0.0605774, -0.0602722, -0.0595093, -0.0590515, -0.0579834, -0.0569153, -0.0550842, -0.0532532, -0.0515747, -0.0498962, -0.0486755, -0.0485229, -0.0491333, -0.0495911, -0.0508118, -0.0527954, -0.0546265, -0.0566101, -0.0579834, -0.0596619, -0.0604248, -0.0602722, -0.0592041, -0.0569153, -0.0549316, -0.0523376, -0.050354, -0.04776, -0.0447083, -0.0401306, -0.0360107, -0.0289917, -0.0204468, -0.00976562, 0.00228882, 0.0160217, 0.0294495, 0.040741, 0.0505066, 0.0572205, 0.0614929, 0.0636292, 0.0630188, 0.0616455, 0.0610352, 0.0614929, 0.0643921, 0.0692749, 0.0762939, 0.085144, 0.0959778, 0.106049, 0.114441, 0.119324, 0.120697, 0.118866, 0.112915, 0.102844, 0.0900269, 0.0762939, 0.0622559, 0.0483704, 0.0352478, 0.0222778, 0.00991821, -0.00183105, -0.0143433, -0.0262451, -0.0389099, -0.0520325, -0.0653076, -0.0778198, -0.0894165, -0.098724

## end test

In [5]:
print(data['hr'])

[[-1.23251646e-02 -1.24299099e-03  6.87468359e-03 -1.33069141e-02
   3.77871677e-03  5.38126340e-03]
 [-7.14022533e-03  2.38219983e-02 -2.40823339e-02 -1.33517513e-02
   3.33270339e-02 -3.71429790e-02]
 [-1.30361924e-02  2.73208844e-02 -1.55380287e-02  2.24158020e-02
  -2.87222212e-02  3.32277981e-02]
 [ 7.73560820e-03 -2.28481948e-02  1.97787817e-02 -7.99560800e-05
   3.75516200e-03  2.81653719e-03]
 [ 3.84237696e-02 -3.02251992e-02  6.20302140e-03 -3.18275300e-04
  -1.41463951e-02  1.21992647e-02]
 [ 1.47366489e-02 -2.37793141e-02  6.99885561e-03 -1.40414152e-02
   2.99704046e-02 -2.75244009e-02]
 [-2.94778244e-02  2.09808506e-02 -1.53140144e-02  2.23437462e-02
  -2.43637085e-02  1.57482863e-02]
 [ 2.46835371e-02 -6.96623928e-03 -1.61187939e-02  1.35833851e-02
  -1.20834169e-02 -2.57044398e-02]
 [ 1.21694732e-02  2.35777340e-03 -3.48878605e-03 -8.83453566e-03
   1.72934473e-02  2.30516123e-03]
 [ 3.52726101e-03 -4.79950648e-03  1.07162574e-02 -2.24639999e-02
  -2.24026233e-02  1.6043

prendo un'implementazione di sppca e la applico al nostro caso

different types of observable data

1) gaze
2) fisio
    2.1) heart rate variabilty
    2.2) eda phasic value
3) social anxiety
4) aspettativa del dolore

gaze =

In [6]:
gaze = pupil_data
hr = hr_data
eda = data['eda']
'''social_anx = data['social anxiety'][:,np.newaxis]
pain_exp = data['pain expectation'][:,np.newaxis]'''


N_gaze = gaze.shape[0]
D_gaze = gaze.shape[1]

N_hr = hr.shape[0]
D_hr = hr.shape[1]

N_eda = eda.shape[0]
D_eda = eda.shape[1]
K = 1


In [7]:
print(D_gaze, D_eda, D_hr)

1 6 1


In [8]:
with pm.Model() as sPPCA:
    #dati osservabili
    gaze_data = pm.MutableData("gaze_data", gaze.T)
    hr_data = pm.MutableData("hr_data", hr.T)
    #eda_data = pm.MutableData("eda_data", eda.T)

    #matrici pesi
    Wgaze = pm.Normal('Wgaze', mu=at.zeros([D_gaze, K]), sigma=2.0 * at.ones([D_gaze, K]), shape=[D_gaze, K])
    Whr = pm.Normal('Whr', mu=at.zeros([D_hr, K]), sigma=2.0 * at.ones([D_hr, K]), shape=[D_hr, K])
    #Weda = pm.Normal('Weda', mu=at.zeros([D_eda, K]), sigma=2.0 * at.ones([D_eda, K]), shape=[D_eda, K])

    #latent space
    c = pm.Normal('c', mu=at.zeros([N_gaze, K]), sigma=at.ones([N_gaze, K]), shape=[N_gaze, K])

    x_gaze = pm.Normal('x_gaze', mu=Wgaze.dot(c.T), sigma=at.ones([D_gaze, N_gaze]), shape=[D_gaze, N_gaze], observed=gaze_data)
    x_hr = pm.Bernoulli('x_hr', p=pm.math.sigmoid(Whr.dot(c.T)), shape=[D_hr, N_hr], observed=hr_data)
    #x_eda = pm.Bernoulli('x_eda', p=pm.math.sigmoid(Weda.dot(c.T)), shape=[D_eda, N_eda], observed=eda_data)

In [9]:
'''gv = pm.model_to_graphviz(sPPCA)
gv.view()'''

'gv = pm.model_to_graphviz(sPPCA)\ngv.view()'

In [10]:
sPPCA.free_RVs

[Wgaze ~ N(f(), f()), Whr ~ N(f(), f()), c ~ N(f(), f())]

In [11]:
with sPPCA:
    trace = pm.sample(init='advi+adapt_diag', chains=1)

Auto-assigning NUTS sampler...
Initializing NUTS using advi+adapt_diag...


ValueError: Input dimension mismatch. One other input has shape[1] = 4, but input[4].shape[1] = 5.
Apply node that caused the error: Elemwise{Composite{Switch(i0, Switch(i1, i2, Switch(i3, i4, i5)), i2)}}(InplaceDimShuffle{x,x}.0, Elemwise{Composite{OR(LT(i0, i1), GT(i0, i2))}}.0, TensorConstant{(1, 1) of -inf}, Elemwise{Cast{int64}}.0, Elemwise{Composite{(-scalar_softplus((-i0)))}}.0, Elemwise{Composite{(-scalar_softplus(i0))}}[(0, 0)].0)
Toposort index: 105
Inputs types: [TensorType(bool, (1, 1)), TensorType(bool, (None, None)), TensorType(float32, (1, 1)), TensorType(int64, (None, None)), TensorType(float64, (1, None)), TensorType(float64, (1, None))]
Inputs shapes: [(1, 1), (1, 4), (1, 1), (1, 4), (1, 5), (1, 5)]
Inputs strides: [(1, 1), (4, 1), (4, 4), (32, 8), (40, 8), (40, 8)]
Inputs values: [array([[ True]]), array([[ True,  True,  True,  True]]), array([[-inf]], dtype=float32), array([[2, 2, 2, 2]]), array([[-0.41666506, -0.21072147, -0.89721847, -0.84046797, -0.77830748]]), a

In [ ]:
#az.plot_posterior(mean_field.sample(1000), color="LightSeaGreen")
az.plot_trace(trace);
plt.show()

In [ ]:
with sPPCA:
    # update values of predictors:
    pm.set_data({"gaze_data": gaze})
    # use the updated values and predict outcomes and probabilities:
    posterior_predictive = pm.sample_posterior_predictive(
        trace, random_seed=123)
hr_pred = posterior_predictive.posterior_predictive["x_hr"]

In [ ]:
print(hr_pred[0])

In [ ]:
hr_pred_mode = np.squeeze(np.mean(hr_pred[0],keepdims=False)[0])[:,np.newaxis]

In [ ]:
train_accuracy_hr = accuracy_score(hr, hr_pred_mode)
train_accuracy_eda = accuracy_score(eda, eda_pred_mode)
print('\n\tAccuracy hr: ' + str(train_accuracy_hr))
print('\tAccuracy eda: ' + str(train_accuracy_eda))
print(' ')

In [ ]:
with sPPCA:
    # update values of predictors:
    pm.set_data({"hr_data": hr, "eda_data": eda})
    # use the updated values and predict outcomes and probabilities:
    posterior_predictive = pm.sample_posterior_predictive(
        trace, var_names=["gaze"], random_seed=123)
    gaze_pred = posterior_predictive["gaze"]

In [ ]:
gaze_pred_mean = np.mean(gaze_pred, axis=0).T.reshape(-1)
gaze_true = gaze.reshape(-1)
reconstructed_r2 = r2_score(gaze_true, gaze_pred_mean)

print('\n\tReconstructed X R2 score: ' + str(reconstructed_r2))

In [ ]:
import deepemogp.feature_extractor as feature_extractor

In [ ]:
f = feature_extractor.FE('wavelet', window=(8,6), params={'w_mother':'db3','w_maxlev':1})

In [ ]:
x = [3, 7, 1, 1, -2, 5, 4, 6]

In [ ]:
f.apply(x)